In [50]:
import pandas as pd
import csv
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import scipy.sparse as sp
airbnb_files = ['data/raw_data/Austin_listings.csv', 'data/raw_data/Boston_listings.csv', 'data/raw_data/Asheville_listings.csv']


In [59]:
q = np.array([1,2,3])
q = q.reshape(q.shape[0], -1)

In [62]:
w = np.array([[1,2],[2,3],[3,4]])
np.hstack((q,w))

array([[1, 1, 2],
       [2, 2, 3],
       [3, 3, 4]])

In [63]:
def drop_airbnb_cols(filename):
    df = pd.read_csv(filename)
    df.drop(['id', 'listing_url', 'scrape_id', 'last_scraped', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url', 'host_thumbnail_url', 'host_picture_url', 'calendar_last_scraped', 'weekly_price', 'monthly_price', 'neighbourhood_cleansed', \
             'license', 'jurisdiction_names', 'square_feet', 'neighbourhood', 'calculated_host_listings_count'], axis=1, inplace=True)
    return df

def get_col_names(files):
    counts = {}
    
    for f in files: 
        for c in drop_airbnb_cols(f).columns: 
            if c in counts: 
                counts[c] += 1
            else: 
                counts[c] = 1

    cols = []
    for c in counts: 
        if counts[c] == 3:
            cols.append(c)
    
    return cols

def segment(vector, train, dev, test): 
    count = len(vector)
    test += vector[-1*int(count*.1):]
    dev += vector[-1*int(count*.2):-1*int(count*.1)]
    train += vector[:int(count*.8)]
    
    return train, dev, test
    
def featurize(df):
    df['price'] = df['price'].map(lambda x: x.replace('$', "").replace(',',""))
    df[['price']] = df[['price']].apply(pd.to_numeric) # turn the price col into a number col
    prices = df['price']
    text_cols = ['neighborhood_overview', 'space', 'description', 'host_about', 'notes', 'summary']
    X = prices.reshape(prices.shape[0], -1)
    for col in text_cols:
        corpus = df[col].fillna(value="").values
        vectorizer = CountVectorizer()
        x = vectorizer.fit_transform(corpus) #TODO: clean text
        X = sp.hstack((X, x))
    np.savez('data/all.sparse',data = x.data ,indices=x.indices,
             indptr =x.indptr, shape=x.shape )
    return X

In [42]:
def create_datasets():
    col_names = get_col_names(airbnb_files)

    train = []
    dev = []
    test = []
    
    for f in airbnb_files:   
        df = pd.read_csv(f, dtype={'zipcode': 'str'})
        df = df[col_names]
        vector = featurize(df)
#         train, dev, test = segment(vector, train, dev, test) # put data from f into train, dev, test
        
    return train, dev, test

In [36]:
def save_datasets(train, dev, test):
    with open('data/train_text.csv', 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(train)

#     with open('data/dev_text.csv', 'w') as csvfile:
#         writer = csv.writer(csvfile)
#         writer.writerows(dev)

#     with open('data/test_text.csv', 'w') as csvfile:
#         writer = csv.writer(csvfile)
#         writer.writerows(test)

In [37]:
a = _

In [128]:
save_datasets(*create_datasets())

X len bf:  5
X len after:  5
CHECK: 
X len bf:  5
X len after:  5
CHECK: 
X len bf:  5
X len after:  5
CHECK: 
12


In [64]:
create_datasets()

shape price:  (5835, 1)
shape x:  (5835, 7096)
shape price:  (5835, 7097)
shape x:  (5835, 11385)
shape price:  (5835, 18482)
shape x:  (5835, 13966)
shape price:  (5835, 32448)
shape x:  (5835, 10526)
shape price:  (5835, 42974)
shape x:  (5835, 6224)
shape price:  (5835, 49198)
shape x:  (5835, 6660)
shape price:  (3585, 1)
shape x:  (3585, 5481)
shape price:  (3585, 5482)
shape x:  (3585, 7477)
shape price:  (3585, 12959)
shape x:  (3585, 10204)
shape price:  (3585, 23163)
shape x:  (3585, 6492)
shape price:  (3585, 29655)
shape x:  (3585, 4764)
shape price:  (3585, 34419)
shape x:  (3585, 5598)
shape price:  (864, 1)
shape x:  (864, 3288)
shape price:  (864, 3289)
shape x:  (864, 5239)
shape price:  (864, 8528)
shape x:  (864, 6400)
shape price:  (864, 14928)
shape x:  (864, 4987)
shape price:  (864, 19915)
shape x:  (864, 3616)
shape price:  (864, 23531)
shape x:  (864, 3015)


([], [], [])

In [65]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    return sp.csr_matrix((  loader['data'], loader['indices'], loader['indptr']),
                         shape = loader['shape'])

In [66]:
load_sparse_csr('data/all.sparse.npz')

<864x3015 sparse matrix of type '<type 'numpy.int64'>'
	with 27405 stored elements in Compressed Sparse Row format>

In [68]:
a = _

In [69]:
a = a.todense()

In [71]:
a[0]

matrix([[0, 0, 0, ..., 0, 0, 0]])